In [98]:
import pandas as pd

tweets = pd.read_csv('data/tweets/processed_all_tweets.csv')

In [101]:
tweets

,datetime,text,token,tokenized_text,filtered_tokenized_text,filtered_text
0,2022-08-29 02:18:35,first few days just 1k but going to 10k if no ...,"['first', 'few', 'days', 'just', '1k', 'but', ...","['first', 'few', 'days', 'just', '1k', 'but', ...","['first', 'days', '1k', 'going', '10k', 'major...",first days 1k going 10k major note 10692 hopef...
1,2022-08-29 02:11:30,good to ask yourself this often,"['good', 'to', 'ask', 'yourself', 'this', 'oft...","['good', 'to', 'ask', 'yourself', 'this', 'oft...","['good', 'ask', 'often']",good ask often
2,2022-08-29 02:11:12,agreed we recently made this change many an in...,"['agreed', 'we', 'recently', 'made', 'this', '...","['agreed', 'we', 'recently', 'made', 'this', '...","['agreed', 'recently', 'made', 'change', 'many...",agreed recently made change many intense effor...
3,2022-08-28 23:51:49,this will happen again just a matter of time,"['this', 'will', 'happen', 'again', 'just', 'a...","['this', 'will', 'happen', 'again', 'just', 'a...","['happen', 'matter', 'time']",happen matter time
4,2022-08-28 22:54:36,the swoop of the x is meant to represent the a...,"['the', 'swoop', 'of', 'the', 'x', 'is', 'mean...","['the', 'swoop', 'of', 'the', 'x', 'is', 'mean...","['swoop', 'x', 'meant', 'represent', 'arc', 'o...",swoop x meant represent arc orbit
...,...,...,...,...,...,...
12958,2018-01-07 03:00:48,near 405,"['near', '405']","['near', '405']","['near', '405']",near 405
12959,2018-01-07 02:33:02,put an old school roller rock restaurant at on...,"['put', 'an', 'old', 'school', 'roller', 'rock...","['put', 'an', 'old', 'school', 'roller', 'rock...","['put', 'old', 'school', 'roller', 'rock', 're...",put old school roller rock restaurant one new ...
12960,2018-01-05 00:30:00,falcon heavy goes vertical,"['falcon', 'heavy', 'goes', 'vertical']","['falcon', 'heavy', 'goes', 'vertical']","['falcon', 'heavy', 'goes', 'vertical']",falcon heavy goes vertical
12961,2018-01-03 08:22:31,a neural net to detect rain no rain or sun are...,"['a', 'neural', 'net', 'to', 'detect', 'rain',...","['a', 'neural', 'net', 'to', 'detect', 'rain',...","['neural', 'net', 'detect', 'rain', 'rain', 's...",neural net detect rain rain sun patient


In [102]:
import gensim
from gensim.utils import simple_preprocess
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])
def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) 
             if word not in stop_words] for doc in texts]

data = tweets.filtered_text.values.tolist()

data_words = list(sent_to_words(data))
# remove stop words
data_words = remove_stopwords(data_words)
print(data_words[:1][0][:30])

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\levit\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['first', 'days', 'going', 'major', 'note', 'hopefully']


In [103]:
import gensim.corpora as corpora
# Create Dictionary
id2word = corpora.Dictionary(data_words)
# Create Corpus
texts = data_words
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
# View
print(corpus[:1][0][:30])

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1)]


In [104]:
from pprint import pprint
# number of topics
num_topics = 10
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.034*"yes" + 0.013*"true" + 0.010*"soon" + 0.010*"thanks" + 0.009*"coming" '
  '+ 0.008*"good" + 0.007*"earth" + 0.007*"year" + 0.006*"also" + '
  '0.006*"starship"'),
 (1,
  '0.010*"sure" + 0.008*"good" + 0.007*"like" + 0.006*"one" + 0.005*"best" + '
  '0.005*"year" + 0.005*"time" + 0.005*"launch" + 0.005*"might" + '
  '0.004*"many"'),
 (2,
  '0.010*"people" + 0.010*"thats" + 0.008*"yeah" + 0.007*"car" + 0.006*"also" '
  '+ 0.006*"day" + 0.005*"model" + 0.005*"time" + 0.005*"way" + '
  '0.005*"interesting"'),
 (3,
  '0.015*"exactly" + 0.008*"even" + 0.008*"great" + 0.007*"time" + '
  '0.007*"work" + 0.007*"team" + 0.006*"would" + 0.005*"maybe" + 0.005*"go" + '
  '0.005*"get"'),
 (4,
  '0.012*"great" + 0.010*"good" + 0.009*"one" + 0.008*"true" + 0.008*"new" + '
  '0.008*"production" + 0.007*"many" + 0.007*"starship" + 0.006*"still" + '
  '0.006*"hard"'),
 (5,
  '0.009*"much" + 0.008*"way" + 0.006*"would" + 0.006*"falcon" + '
  '0.006*"probably" + 0.005*"life" + 0.005*"next" + 

In [105]:
lda_model.topics_values()

AttributeError: 'LdaMulticore' object has no attribute 'topics_values'